# RTA Serial Plotter

This notebook will help you record, and live plot the temperature of the RTA over time.
 
> Note that once all heating steps are complete, no more temperature data can be read. The reading process may terminate before the actual specified number of seconds to read for.


**Before pressing the START button on the RTA, run the following cells:**

1. Import required packages
2. Define where data is saved
3. Initialize plot

In [1]:
from pathlib import Path 
import h5py
import numpy as np
import copy
import plotly.graph_objs as go
import matplotlib.pyplot as plt 

import SerialReader
from SerialReader import SerialReaderPlotter

data_path = Path(r"RTA-data\2024-02-09_")                # Where data is saved
data_path.mkdir(parents=True, exist_ok=True)

fig = go.FigureWidget(data=[])
fig.update_layout(
    template="plotly_white",
    xaxis_title="Time (s)",
    yaxis_title="Temperature (C)",
)

FigureWidget({
    'data': [],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'T…

5. Define total time of plotting data
6. Initialize serial connection
7. Begin plotting & writing threads

In [2]:
READ_FOR = 60*60*6       # Total time (s) the serial reader will plot

port = 'COM3'           # Find this port in Device Manager

s = SerialReaderPlotter(port, READ_FOR, fig, baudrate=19200, timeout=1)


s.start_reading()
s.start_plotting()

(Reader   ) Starting
(Plotter  ) Starting


In [3]:
fig.show()
#fig.write_html(r'C:\Users\Lab Member\Desktop\SerialPlotOutput.html')

Scroll up and you should see that the plot is updating now.

8. **Press START** on the RTA to begin annealing.


# Saving Data

In [6]:
COMMENT = "RTA_clean"

In [7]:
from datetime import datetime

data_path = Path(f"RTA-data\{datetime.now().date()}")                # Where data is saved
data_path.mkdir(parents=True, exist_ok=True)

date = datetime.now().strftime("%Y_%m_%d-%I-%M_%p")
print("Today: {}".format(date))

Today: 2024_02_09-02-32_PM


In [8]:
fig = copy.copy(fig)
for i, scatter in enumerate(fig.data):
    filename = data_path/Path("RTA_Run_{}_".format(i) + COMMENT + "_{}.h5".format(date))
    with h5py.File(filename, 'w') as hf:
        hf.create_dataset('time_arr', data=np.array(scatter.x[1:], dtype=float)) 
        hf.create_dataset('temperature_arr', data=np.array(scatter.y[1:], dtype=float))

## Loading Data

In [ ]:
# Specify which directory the data will be read from
read_path = Path(r"RTA-data\2022-10-17")

In [ ]:
previous_runs = {f.name: h5py.File(f, "r") for f in read_path.glob("*.h5")}

In [7]:
read = go.FigureWidget(data=[])
read.update_layout(
    template="plotly_white",
    xaxis_title="Time (s)",
    yaxis_title="Temperature (C)",
)

for k in previous_runs.keys():
    read.add_scatter(x=[], 
                    y=[], 
                    line=dict(width=1))

    read_dat = read.data[-1]
    
    x = previous_runs[k]["time_arr"][:]
    y = previous_runs[k]["temperature_arr"][:]
    with read.batch_update():
        read_dat.x += tuple(x)
        read_dat.y += tuple(y) 
    previous_runs[k].close()
read

NameError: name 'previous_runs' is not defined

In [ ]:
# Now we want to aliogn to a temperature so we can compare the PID values better
previous_runs = {f.name: h5py.File(f, "r") for f in read_path.glob("*.h5")}

temp_align = 300


read = go.FigureWidget(data=[])
read.update_layout(
    template="plotly_white",
    xaxis_title="Time (s)",
    yaxis_title="Temperature (C)",
)

for k in previous_runs.keys():
    read.add_scatter(x=[], 
                    y=[], 
                    line=dict(width=1),
                    name=f'{k}')

    read_dat = read.data[-1]
    
    x_raw = previous_runs[k]["time_arr"][:]
    y_raw = previous_runs[k]["temperature_arr"][:]

    index = np.where(y_raw > temp_align)[0][0]
    x = x_raw[index:] - x_raw[index]
    y = y_raw[index:]
    
    with read.batch_update():
        read_dat.x += tuple(x)
        read_dat.y += tuple(y) 
    previous_runs[k].close()

# read.update_layout(hovermode='x unified')
read